In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# 그래프에서 한글 깨짐 방지
import platform
if platform.system() == "Darwin":          # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

In [55]:
df = pd.read_csv('../상장기업/Dataset/t-1_raw_상장.csv')

df['FCF'] = df['OCF(천원)'] + df['ICF(천원)']
df['매출액대비잉여현금흐름'] = df['FCF'] / df['매출액(천원)']
df['총자산대비영업현금흐름'] = df['OCF(천원)'] / df['자산총계(천원)']
df = df[['회사명', '거래소코드', '부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름', '자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률', 't-1감사의견코드']]
df = df[['회사명', '거래소코드', '부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름', '자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률', 't-1감사의견코드']]

In [56]:
df.isna().sum()
df.dropna(axis=0,inplace=True)
df.isin([np.inf,-np.inf]).sum()
df = df.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 대체
df = df.dropna()  # NaN이 있는 행 삭제

In [57]:
import numpy as np

# 컬럼 이름
column_name = '매출액영업이익률'

# 하위 1%, 상위 1% 값을 계산
lower_bound = df[column_name].quantile(0.005)
upper_bound = df[column_name].quantile(0.995)

# 1% 미만 및 1% 초과 데이터 선택 후 삭제
df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]


In [58]:
# Scaling
scaler=MinMaxScaler()
df.iloc[:,2:-1]=scaler.fit_transform(df.iloc[:,2:-1])

In [59]:
# dp-score 계수
coefficients = {
    '부채비율' : np.exp(-0.30259864),
    '총자본회전률' : np.exp(1.0667429),
    '매출액대비잉여현금흐름' : np.exp(0.03715004),
    'PBR' : np.exp(-0.33048908),
    '총자산대비영업현금흐름' : np.exp(-0.12926251),
    '자기자본증가율' : np.exp(-0.36731349),
    '총자본투자효율' : np.exp(-0.33997948),
    '총자본순이익률' : np.exp(0.94653419),
    '매출액영업이익률' : np.exp(-0.35456208),
    'constant' : 0.71344978
}

# O-Score 계산
df['dp_score'] = coefficients['constant']
df['dp_score'] += coefficients['부채비율'] * df['부채비율']
df['dp_score'] += coefficients['총자본회전률'] * df['총자본회전률']
df['dp_score'] += coefficients['매출액대비잉여현금흐름'] * df['매출액대비잉여현금흐름']
df['dp_score'] += coefficients['PBR'] * df['PBR']
df['dp_score'] += coefficients['총자산대비영업현금흐름'] * df['총자산대비영업현금흐름']
df['dp_score'] += coefficients['자기자본증가율'] * df['자기자본증가율']
df['dp_score'] += coefficients['총자본투자효율'] * df['총자본투자효율']
df['dp_score'] += coefficients['총자본순이익률'] * df['총자본순이익률']
df['dp_score'] += coefficients['매출액영업이익률'] * df['매출액영업이익률']

In [60]:
df_상장=df[df['t-1감사의견코드']==1]
# df_상장['dp_score'].mean()

---

In [61]:
df=pd.read_csv('../외감기업/raw_data/t-1_raw_외감.csv',encoding='euc-kr')
df['자산총계(천원)'] = df['자산총계(천원)'] * 1000
df['log자산총계'] = np.log(df['자산총계(천원)'])
df['잉여현금흐름(FCF)']=df['OCF(천원)']-df['ICF(천원)']
df['전체현금흐름(천원)']=df['OCF(천원)']+df['ICF(천원)']+df['FCF(천원)']

df['총자산대비잉여현금흐름'] = df['잉여현금흐름(FCF)']/df['자산총계(천원)']
df['총자산대비현금흐름'] = df['전체현금흐름(천원)'] / df['자산총계(천원)']
df=df[['유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율','t-1감사의견코드']]

In [62]:
df.isna().sum()
df.dropna(axis=0,inplace=True)
df.isin([np.inf,-np.inf]).sum()
df = df.replace([np.inf, -np.inf], np.nan)  # inf 값을 NaN으로 대체
df = df.dropna()  # NaN이 있는 행 삭제

In [63]:
import numpy as np

# 컬럼 이름
column_name = '자기자본증가율'

# 하위 1%, 상위 1% 값을 계산
lower_bound = df[column_name].quantile(0.005)
upper_bound = df[column_name].quantile(0.995)

# 1% 미만 및 1% 초과 데이터 선택 후 삭제
df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

# 컬럼 이름
column_name = '유동자산회전률'

# 하위 1%, 상위 1% 값을 계산
lower_bound = df[column_name].quantile(0.005)
upper_bound = df[column_name].quantile(0.995)

# 1% 미만 및 1% 초과 데이터 선택 후 삭제
df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

# 컬럼 이름
column_name = '총자산대비현금흐름'

# 하위 1%, 상위 1% 값을 계산
lower_bound = df[column_name].quantile(0.005)
upper_bound = df[column_name].quantile(0.995)

# 1% 미만 및 1% 초과 데이터 선택 후 삭제
df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

In [64]:
scaler=MinMaxScaler()
df.iloc[:,0:10]=scaler.fit_transform(df.iloc[:,0:10])

In [65]:
# dp-Score의 계수
coefficients = {
    '유동자산회전률':np.exp(0.47765086),                    # 높아야 좋다
    '총자산대비잉여현금흐름':np.exp(0.53785436),            # 높아야 좋다
    '자기자본구성비율':np.exp(0.49360789),                  # 높아야 좋다
    'log자산총계':np.exp(0.94238478),                      # 높아야 좋다
    '자기자본회전률':np.exp(-0.08646556),                   # 높아야 좋다
    '순운전자본회전률':np.exp(-0.98107367),                 # 높아야 좋다
    '자기자본증가율':np.exp(0.19544705),                    # 높아야 좋다
    '총자본증가율':np.exp(0.44094258),                      # 높아야 좋다
    '총자산대비현금흐름':np.exp(-0.37036301),               # 높아야 좋다
    '총자본투자효율':np.exp(0.26458857),                    # 높아야 좋다

    'constant': 0.90495308
}


# O-Score 계산
df['dp_score'] = coefficients['constant']
df['dp_score'] += coefficients['유동자산회전률'] * df['유동자산회전률']
df['dp_score'] += coefficients['총자산대비잉여현금흐름'] * df['총자산대비잉여현금흐름']
df['dp_score'] += coefficients['자기자본구성비율'] * df['자기자본구성비율']
df['dp_score'] += coefficients['log자산총계'] * df['log자산총계']
df['dp_score'] += coefficients['자기자본회전률'] * df['자기자본회전률']
df['dp_score'] += coefficients['순운전자본회전률'] * df['순운전자본회전률']
df['dp_score'] += coefficients['자기자본증가율'] * df['자기자본증가율']
df['dp_score'] += coefficients['총자본증가율'] * df['총자본증가율']
df['dp_score'] += coefficients['총자산대비현금흐름'] * df['총자산대비현금흐름']
df['dp_score'] += coefficients['총자본투자효율'] * df['총자본투자효율']

# 로지스틱 함수를 이용해 부도 확률 계산
# df_등급화['default_probability'] = 1 / (1 + np.exp(-df_등급화['dp_score']))

In [68]:
df_외감 =df[df['t-1감사의견코드']==1]
df_외감['dp_score'].mean()

4.4141268217511875

---

- dp_score의 상장 , 비상장 외감기업의 레이블링이 1인 기업들의 유의미한 차이가 있는지 검정

- 1. 등분산 검정 (levene,bartlett)
- 2. t-test검정  (Welch's t-test)

---

In [70]:
import numpy as np
from scipy.stats import levene

# 두 그룹의 데이터 생성

# Levene의 등분산성 검정 수행
statistic, p_value = levene(df_상장['dp_score'], df_외감['dp_score'])

# 결과 출력
print("Levene's 등분산성 검정 결과:")
print("통계량:", statistic)
print("p-value:", p_value)

# 유의수준 설정
alpha = 0.05

# p-value와 유의수준을 비교하여 결과 해석
if p_value < alpha:
    print("p-value가 유의수준보다 작으므로 등분산성 가정을 기각합니다.")
    print("따라서 등분산성이 만족되지 않습니다.")
else:
    print("p-value가 유의수준보다 크므로 등분산성 가정을 기각할 수 없습니다.")
    print("따라서 등분산성이 만족됩니다.")


Levene's 등분산성 검정 결과:
통계량: 27.797153413031545
p-value: 1.3685239738560248e-07
p-value가 유의수준보다 작으므로 등분산성 가정을 기각합니다.
따라서 등분산성이 만족되지 않습니다.


In [72]:
import numpy as np
from scipy.stats import bartlett

# Bartlett의 등분산성 검정 수행
statistic, p_value = bartlett(df_상장['dp_score'], df_외감['dp_score'])

# 결과 출력
print("Bartlett's 등분산성 검정 결과:")
print("통계량:", statistic)
print("p-value:", p_value)

# 유의수준 설정
alpha = 0.05

# p-value와 유의수준을 비교하여 결과 해석
if p_value < alpha:
    print("p-value가 유의수준보다 작으므로 등분산성 가정을 기각합니다.")
    print("따라서 등분산성이 만족되지 않습니다.")
else:
    print("p-value가 유의수준보다 크므로 등분산성 가정을 기각할 수 없습니다.")
    print("따라서 등분산성이 만족됩니다.")


Bartlett's 등분산성 검정 결과:
통계량: 32.2777488813525
p-value: 1.336349264112477e-08
p-value가 유의수준보다 작으므로 등분산성 가정을 기각합니다.
따라서 등분산성이 만족되지 않습니다.


---

In [71]:
import numpy as np
from scipy.stats import ttest_ind

# 두 그룹의 데이터 생성

# Welch's t-test 수행
t_statistic, p_value = ttest_ind(df_상장['dp_score'], df_외감['dp_score'], equal_var=False)

# 결과 출력
print("Welch's t-test 결과:")
print("t-statistic:", t_statistic)
print("p-value:", p_value)

# 유의수준 설정
alpha = 0.05

# p-value와 유의수준을 비교하여 결과 해석
if p_value < alpha:
    print("p-value가 유의수준보다 작으므로 귀무가설을 기각합니다.")
    print("따라서 두 그룹 간의 평균 차이는 통계적으로 유의미합니다.")
else:
    print("p-value가 유의수준보다 크므로 귀무가설을 기각할 수 없습니다.")
    print("따라서 두 그룹 간의 평균 차이는 통계적으로 유의미하지 않습니다.")


Welch's t-test 결과:
t-statistic: 30.440984262331284
p-value: 8.923973556398974e-44
p-value가 유의수준보다 작으므로 귀무가설을 기각합니다.
따라서 두 그룹 간의 평균 차이는 통계적으로 유의미합니다.
